In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.model_selection import train_test_split

In [22]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
import torch.nn as nn
import torch.optim as optim

# Define payment methods and features
payment_methods = ["Amazon Pay UPI", "Google Pay", "PhonePe", "Credit Card", "Debit Card", "EMI", "Net Banking", "Cash on Delivery"]
num_payment_methods = len(payment_methods)

# User types in cyclic order
user_types = ["cashback_oriented", "cost_oriented", "history_oriented"]

# Function to generate user data
def generate_user_data(num_users):
    data = []
    labels = []
    user_payment_history = {user_id: {method: 0 for method in payment_methods} for user_id in range(1, num_users + 1)}
    payment_success_rates = {method: {"success": 0, "total": 0} for method in payment_methods}

    for user_id in range(1, num_users + 1):
        user_type_index = 0  # Start with the first user type
        for product_id in range(1, 11):  # Assuming each user orders 10 products
            price = np.random.randint(100, 10000)  # Random price between 100 and 10000
            payment_options = []
            
            user_type = user_types[user_type_index % len(user_types)]
            for method in payment_methods:
                cost = np.random.uniform(0, 0.05)   # Cost as percentage of the product price
                cashback = np.random.uniform(0, 0.10)  # Cashback as percentage of the product price
                
                # Calculate current success rate based on past transactions
                success = payment_success_rates[method]["success"]
                total = payment_success_rates[method]["total"]
                success_rate = (success / total) if total > 0 else np.random.uniform(0.70, 0.99)
    
                user_history = min(user_payment_history[user_id][method], 10)  # User history (how many times used in recent transactions)
                if user_type == "cashback_oriented":
                    cost = 0 
                    user_history = 0 
                elif user_type == "history_oriented":
                    cost = 0
                    cashback = 0 
                else:
                    user_history = 0 
                    cashback = 0 
                payment_options.append([cost, cashback, success_rate, user_history])
  # Determine the current user type in cyclic order

            # Determine the best payment method based on user type
            temp = [0] * len(payment_methods)
            if user_type == "cashback_oriented":
                best_method_index = max(range(num_payment_methods), key=lambda i: payment_options[i][1])
                
            elif user_type == "history_oriented":
                best_method_index = max(range(num_payment_methods), key=lambda i: payment_options[i][3])
            elif user_type == "cost_oriented":
                best_method_index = min(range(num_payment_methods), key=lambda i: payment_options[i][0])
            temp[best_method_index] = 1 
            data.append(payment_options)
#             labels.append(best_method_index)
            labels.append(temp)

            # Update user payment history and success rates after generating data for each product
            for i, method in enumerate(payment_methods):
                if i == best_method_index:
                    user_payment_history[user_id][method] += 1
                    if np.random.rand() > 0.05:  # Assuming a 95% success rate for each transaction
                        payment_success_rates[method]["success"] += 1
                    payment_success_rates[method]["total"] += 1

            user_type_index += 1  # Move to the next user type for the next product

    return np.array(data), np.array(labels)

# Number of users
num_users = 100

# Generate data for users
X, y = generate_user_data(num_users)

# Convert to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

# Create a dataset and data loader
dataset = TensorDataset(X_tensor, y_tensor)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [23]:
X.shape , y.shape

((1000, 8, 4), (1000, 8))

In [24]:
for a , b in train_loader : 
    print(a.shape)
    print(b.shape)
    break

torch.Size([32, 8, 4])
torch.Size([32, 8])


In [25]:
class PaymentMethodCNN(nn.Module):
    def __init__(self):
        super(PaymentMethodCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=(3, 3), padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=(3, 3), padding=1)
        self.fc1 = nn.Linear(64 * 8 , 128)
        self.fc2 = nn.Linear(128, 8 )
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout = nn.Dropout(0.25)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = x.unsqueeze(1)  # Add channel dimension
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.sigmoid(self.fc2(x))
        return x

# Initialize the model, loss function, and optimizer
model = PaymentMethodCNN()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [26]:
# Training function
def train_model(model, train_loader, criterion, optimizer, num_epochs=20):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
        epoch_loss = running_loss / len(train_loader.dataset)
        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}')

# Evaluation function
def evaluate_model(model, test_loader, criterion):
    model.eval()
    total = 0
    running_loss = 0.0
    correct_predictions = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            predicted_labels = torch.argmax(outputs, dim=1)
            true_labels = torch.argmax(labels, dim=1)
            correct_predictions += (predicted_labels == true_labels).sum().item()
            total += labels.size(0)
    accuracy = correct_predictions / total
    avg_loss = running_loss / len(test_loader.dataset)
    return avg_loss, accuracy


In [27]:

# Train the model
train_model(model, train_loader, criterion, optimizer, num_epochs=20)

# Evaluate the model
test_loss, test_accuracy = evaluate_model(model, test_loader, criterion)
print(f'Test Loss: {test_loss:.4f}')


Epoch 1/20, Loss: 0.1471
Epoch 2/20, Loss: 0.1150
Epoch 3/20, Loss: 0.1109
Epoch 4/20, Loss: 0.1070
Epoch 5/20, Loss: 0.1035
Epoch 6/20, Loss: 0.0966
Epoch 7/20, Loss: 0.0926
Epoch 8/20, Loss: 0.0896
Epoch 9/20, Loss: 0.0873
Epoch 10/20, Loss: 0.0847
Epoch 11/20, Loss: 0.0835
Epoch 12/20, Loss: 0.0822
Epoch 13/20, Loss: 0.0819
Epoch 14/20, Loss: 0.0809
Epoch 15/20, Loss: 0.0804
Epoch 16/20, Loss: 0.0802
Epoch 17/20, Loss: 0.0801
Epoch 18/20, Loss: 0.0790
Epoch 19/20, Loss: 0.0791
Epoch 20/20, Loss: 0.0781
Test Loss: 0.0799


In [28]:

# Train the model
train_model(model, train_loader, criterion, optimizer, num_epochs=20)

# Evaluate the model
test_loss, test_accuracy = evaluate_model(model, test_loader, criterion)
print(f'Test Loss: {test_loss:.4f}')


Epoch 1/20, Loss: 0.0785
Epoch 2/20, Loss: 0.0784
Epoch 3/20, Loss: 0.0785
Epoch 4/20, Loss: 0.0777
Epoch 5/20, Loss: 0.0768
Epoch 6/20, Loss: 0.0771
Epoch 7/20, Loss: 0.0760
Epoch 8/20, Loss: 0.0761
Epoch 9/20, Loss: 0.0765
Epoch 10/20, Loss: 0.0756
Epoch 11/20, Loss: 0.0751
Epoch 12/20, Loss: 0.0745
Epoch 13/20, Loss: 0.0744
Epoch 14/20, Loss: 0.0734
Epoch 15/20, Loss: 0.0729
Epoch 16/20, Loss: 0.0725
Epoch 17/20, Loss: 0.0724
Epoch 18/20, Loss: 0.0723
Epoch 19/20, Loss: 0.0710
Epoch 20/20, Loss: 0.0707
Test Loss: 0.0719


In [29]:

# Train the model
train_model(model, train_loader, criterion, optimizer, num_epochs=20)

# Evaluate the model
test_loss, test_accuracy = evaluate_model(model, test_loader, criterion)
print(f'Test Loss: {test_loss:.4f}')


Epoch 1/20, Loss: 0.0701
Epoch 2/20, Loss: 0.0688
Epoch 3/20, Loss: 0.0694
Epoch 4/20, Loss: 0.0688
Epoch 5/20, Loss: 0.0679
Epoch 6/20, Loss: 0.0683
Epoch 7/20, Loss: 0.0668
Epoch 8/20, Loss: 0.0667
Epoch 9/20, Loss: 0.0661
Epoch 10/20, Loss: 0.0647
Epoch 11/20, Loss: 0.0635
Epoch 12/20, Loss: 0.0639
Epoch 13/20, Loss: 0.0621
Epoch 14/20, Loss: 0.0612
Epoch 15/20, Loss: 0.0607
Epoch 16/20, Loss: 0.0609
Epoch 17/20, Loss: 0.0590
Epoch 18/20, Loss: 0.0588
Epoch 19/20, Loss: 0.0571
Epoch 20/20, Loss: 0.0549
Test Loss: 0.0598


In [30]:

def predict_best_payment_method(model, price, payment_methods_data):
    """
    Predicts the best payment method based on the input data.

    Args:
    - model (nn.Module): Trained CNN model for payment method prediction.
    - price (float): Price of the product.
    - payment_methods_data (list of dict): List of dictionaries containing data for each payment method.
      Each dictionary should have keys: 'cost', 'cashback', 'success_rate', 'user_history'.

    Returns:
    - best_payment_method (str): The predicted best payment method.
    """
    # Validate input data
    assert len(payment_methods_data) == len(payment_methods), "Data for each payment method must be provided."

    # Prepare input tensor
    input_data = []
    for method_data in payment_methods_data:
        input_data.append([ method_data['cost'], method_data['cashback'], method_data['success_rate'], method_data['user_history']])
    input_tensor = torch.tensor([input_data], dtype=torch.float32)

    # Model prediction
    model.eval()
    with torch.no_grad():
        output = model(input_tensor)
#         predicted_index = torch.argmax(output, dim=1).item()
    return output
    # Get the best payment method
    best_payment_method = payment_methods[predicted_index]
    
    return best_payment_method

In [31]:
price = 1500.0

# Define the payment methods data
payment_methods = ['Amazon Pay UPI', 'Google Pay', 'PhonePe', 'Credit Card', 'Debit Card', 'EMI', 'Net Banking', 'Cash on Delivery']
payment_methods_data = [
    {'cost': 0.02 , 'cashback': 0.05  , 'success_rate': 0.95, 'user_history': 5},  # Amazon Pay UPI
    {'cost': 0.015 , 'cashback': 0.04  , 'success_rate': 0.92, 'user_history': 4},  # Google Pay
    {'cost': 0.025 , 'cashback': 0.03  , 'success_rate': 0.90, 'user_history': 1},  # PhonePe
    {'cost': 0.03 , 'cashback': 0.02  , 'success_rate': 0.85, 'user_history': 0},  # Credit Card
    {'cost': 0.02 , 'cashback': 0.01  , 'success_rate': 0.88, 'user_history': 0},  # Debit Card
    {'cost': 0.04 , 'cashback': 0.03 , 'success_rate': 0.89, 'user_history': 0},  # EMI
    {'cost': 0.01 , 'cashback': 0.02 , 'success_rate': 0.87, 'user_history': 0},  # Net Banking
    {'cost': 0.004 , 'cashback': 0.0, 'success_rate': 0.80, 'user_history': 0},    # Cash on Delivery
]

# Initialize the trained model (assuming the model is already trained)
# model = PaymentMethodCNN()
# model.load_state_dict(torch.load('path_to_trained_model.pth'))

# Get the best payment method
best_payment_method = predict_best_payment_method(model, price, payment_methods_data)
# print(f"The best payment method is: {best_payment_method}")
print(best_payment_method)

tensor([[1.0000e+00, 1.4264e-12, 1.1452e-38, 1.7047e-36, 1.6453e-33, 4.8536e-35,
         4.3571e-33, 1.4080e-16]])


In [32]:
price = 1500.0

# Define the payment methods data
payment_methods = ['Amazon Pay UPI', 'Google Pay', 'PhonePe', 'Credit Card', 'Debit Card', 'EMI', 'Net Banking', 'Cash on Delivery']
payment_methods_data = [
    {'cost': 0.02 , 'cashback': 0.05  , 'success_rate': 0.95, 'user_history': 1},  # Amazon Pay UPI
    {'cost': 0.015 , 'cashback': 0.14 , 'success_rate': 0.92, 'user_history': 1},  # Google Pay
    {'cost': 0.025 , 'cashback': 0.03  , 'success_rate': 0.90, 'user_history': 1},  # PhonePe
    {'cost': 0.03 , 'cashback': 0.02  , 'success_rate': 0.85, 'user_history': 1},  # Credit Card
    {'cost': 0.02 , 'cashback': 0.01  , 'success_rate': 0.88, 'user_history': 1},  # Debit Card
    {'cost': 0.04 , 'cashback': 0.03 , 'success_rate': 0.89, 'user_history': 1},  # EMI
    {'cost': 0.01 , 'cashback': 0.02 , 'success_rate': 0.87, 'user_history': 1},  # Net Banking
    {'cost': 0.004 , 'cashback': 0.0, 'success_rate': 0.80, 'user_history': 1},    # Cash on Delivery
]

# Initialize the trained model (assuming the model is already trained)
# model = PaymentMethodCNN()
# model.load_state_dict(torch.load('path_to_trained_model.pth'))

# Get the best payment method
best_payment_method = predict_best_payment_method(model, price, payment_methods_data)
# print(f"The best payment method is: {best_payment_method}")
print(list(best_payment_method))

[tensor([3.3522e-01, 4.4153e-01, 1.7158e-05, 1.1191e-06, 5.8606e-06, 1.0774e-06,
        1.4464e-05, 2.0715e-04])]


In [33]:
torch.save(model , "cnn_recommendation_model.pth")